In [2]:
import os
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [79]:
stops = stopwords.words("english")

tfidf = TfidfVectorizer(
    analyzer="word",
    stop_words="english"
)

<class 'list'>


In [41]:
path_to_articles = "screenplay_data\data/raw_text_lemmas/raw_text_lemmas"
articles = [os.path.join(path_to_articles, item) 
            for item in os.listdir(path_to_articles)]

In [77]:
articles_texts = []
titles = []

for article_path in articles:
    titles.append(article_path)
    with open(article_path, "r", encoding="utf-8") as a_src:
        # первые 6 строк — метаданные, последняя — число комментариев
        articles_texts.append("\n".join(a_src.readlines()))

In [22]:
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [26]:
from tqdm import tqdm

In [45]:
articles_preprocessed = []
for a_text in tqdm(articles_texts):
    # for lett in a_text:
    #     if lett.isalpha():
    #         continue
    #     else:
    #         a_text.replace(lett, '')

    try:
        a_tokens = wordpunct_tokenize(a_text)
        a_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in a_tokens])
        articles_preprocessed.append(a_lemmatized)
    except:
        print("well, trouble")

 41%|████      | 1169/2858 [08:20<37:22,  1.33s/it]

well, trouble


 67%|██████▋   | 1901/2858 [14:31<49:19,  3.09s/it]

well, trouble


100%|██████████| 2858/2858 [22:48<00:00,  2.09it/s]


In [69]:
articles_preprocessed_another = []
for a_text in tqdm(articles_texts[:20]):
    # for lett in a_text:
    #     if lett.isalpha():
    #         continue
    #     else:
    #         a_text.replace(lett, '')

    try:
        a_tokens = wordpunct_tokenize(a_text)
        a_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in a_tokens])
        articles_preprocessed_another.append(a_lemmatized)
    except:
        print("well, trouble")

100%|██████████| 20/20 [00:08<00:00,  2.42it/s]


In [47]:
articles_tfidf = tfidf.fit_transform(articles_preprocessed)
print(f"Матрица на {articles_tfidf.shape[0]} документов и {articles_tfidf.shape[1]} термов")

Матрица на 2856 документов и 338738 термов


In [70]:
articles_tfidf_another = tfidf.fit_transform(articles_preprocessed_another)
print(f"Матрица на {articles_tfidf_another.shape[0]} документов и {articles_tfidf_another.shape[1]} термов")

Матрица на 20 документов и 17843 термов


In [48]:
import numpy as np
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

In [50]:
feature_names = np.array(tfidf.get_feature_names())

for i, article in enumerate(articles_texts):
    # напечатаю только первые 5 статей
    if i < 5:
        article_vector = articles_tfidf[i, :]
        words = get_top_tf_idf_words(article_vector, feature_names, 10)
        # print(article)
        print(words)

['michelle' 'howard' 'nate' 'hazmat' 'door' 'room' 'crutch' 'cont' 'int'
 'look']
['kat' 'bianca' 'patrick' 'cameron' 'mandella' 'joey' 'michael' 'walter'
 'chastity' 'perky']
['jasper' 'pongo' 'anita' 'horace' 'cruella' 'perdy' 'cu' 'roger' 'kipper'
 'puppy']
['juror' '8th' '3rd' 'foreman' '4th' 'th' '7th' '11th' '9th' '6th']
['railly' 'cole' 'jeffrey' 'mason' 'dr' 'astrophysicist' 'fale' 'james'
 'look' 'microbiologist']


теперь по хорошему надо кластеризовать

In [53]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [73]:
X = articles_tfidf_another
feature_names = tfidf.get_feature_names()
model = KMeans(n_clusters=10, random_state=1)
model.fit(X)

C:\Users\Andrey Chirkin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


KMeans(n_clusters=10, random_state=1)

In [74]:
dense = X.todense()
denselist = dense.tolist()
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidf.get_feature_names()

C:\Users\Andrey Chirkin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [76]:
with open("output_cluster.txt", "w", encoding="utf-8") as f:
    for i in range(10):
        f.write(f"Cluster {i}\n")
        for j in order_centroids[i, :10]:
            f.write(' %s' % terms[j])
            f.write('\n')
        f.write('\n\n')